In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, AutoModelForSequenceClassification
from peft import PeftModel

tokenizer = AutoTokenizer.from_pretrained("./bloom-1b1")
ipt = tokenizer("Human: {}\n{}".format("How to prepare an exam?", "").strip() + "\n\nAssistant: ", return_tensors="pt")

In [2]:
#bitfit

model = AutoModelForCausalLM.from_pretrained("./bitfit/chatbot/checkpoint-1000")
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)

'Human: How to prepare an exam?\n\nAssistant: Before reading the answers here, it is important to note that a practical exam is not meant to be completed without any preparation. For example, the answer to a practical question may not be the same as the question given in a paper. The exam should be considered to be an assessment of your analytical abilities. It would be up to you to choose the right exam system, study method, and prepare for the exam beforehand.'

In [3]:
#prompt tuning

#peft_model = PeftModel.from_pretrained(model=model, model_id="./prompt_tuning/chatbot/checkpoint-1000/")
#print(tokenizer.decode(peft_model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True))

In [4]:
#prefix tuning

model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-1b1", low_cpu_mem_usage=True)
peft_model = PeftModel.from_pretrained(model=model, model_id='./prefix_tuning/chatbot/checkpoint-1000')
# tokenizer = AutoTokenizer.from_pretrained('prefix-tuning-model')
peft_model = peft_model.cuda()
ipt = tokenizer("Human: {}\n{}".format("How to prepare an exam?", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(peft_model.device)
tokenizer.decode(peft_model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)

'Human: How to prepare an exam?\n\nAssistant: In order to prepare a test you must first set a time, schedule and date for it. This can be done by accessing the test schedule from the exam syllabus, by submitting an exam assignment on the assessment page or by requesting a test invitation from the test host. Once the time, schedule and date have been selected, it is time to formulate and complete a test question. In order for the question to be answered correctly, you must understand the question and answer each step thoroughly and be prepared to answer correctly. By completing the question, you have a clear understanding of the topics'

In [6]:
#lora

peft_model = PeftModel.from_pretrained(model=model, model_id='./lora/chatbot/checkpoint-1000')
peft_model = peft_model.cuda()
ipt = tokenizer("Human: {}\n{}".format("How to prepare an exam?", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(peft_model.device)
tokenizer.decode(peft_model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)

'Human: How to prepare an exam?\n\nAssistant: Understanding the main skills needed to prepare for a certification exam varies based on the type of certification you are aiming to obtain, but these can often be grouped into two main parts: 1) Understanding the key concepts and core skills required for passing an exam and 2) Implementing these elements to achieve an optimal level of performance. For example, there are many different forms of certification exams, such as Certificate in Computer Science (CS), Certificate in Business Systems (CABS), License in Information Technology (LIT), and Master of Information Systems (MIS). Each of these forms of certification'